In [14]:
#Loading dataset

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as pl
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import re
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
gender=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


**Preprocessing the data**
* checking the nature of the data
* viewing the first 8 rows of the train data
* Handling the null value in the age,embarked and the cabin columns using a function handle_null
* Checking for duplicates
* Feature Engineering->Extracting the name column to names only and tickets to numeric only.
* Encoding Categorical Data.


In [15]:
print(train.info())
train.head(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [16]:
#checking null value
print(train.isnull().sum())
print(test.isnull().sum())



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


**Preprocessing the data**# 
Handling the null values

In [17]:
def handle_null(df):
    '''
    cleans data by replacing the null values
    in  Age it replaces the null with  median
    in Cabin it replaces the null with unknown
    in Embarked it replaces the null with mode
    '''
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Cabin'].fillna('Unknown', inplace=True)
    return df
    
train=handle_null(train)
test=handle_null(test)

train


<ipython-input-17-01046093c2df>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-17-01046093c2df>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Unknown,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,Unknown,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [10]:
print(train.isnull().sum())
print(test.isnull().sum())

NameError: name 'train' is not defined

In [ ]:
#checking for duplicates
dups=train.duplicated().sum()
if dups>0:
    print('Duplicate available')

else:
    print('No duplicates')


In [ ]:
#Feature Engineering.

#load the spacy English

nlp= spacy.load('en_core_web_sm')

def preprocess_name(name):
    #convert to lower case
    name=name.lower()
    #remove titles
    name= re.sub(r'(mr\.|mrs\.|miss|dr\.|master)','',name).strip()
    #apply spacy NLP processing
    doc=nlp(name)
    #extract the name only
    Extracted_name = " ".join([token.lemma_ for token in doc if token.ent_type_ == "PERSON" or token.is_alpha])
   
    
    return Extracted_name

def preprocess_ticket(ticket):
    ticket_str=str(ticket)
    #exacting the numeric value of ticket
    Ticket_new =''.join(filter(str.isdigit,ticket_str))
    
    return Ticket_new



#apply the preprocessing function

train['Extracted_name']=train['Name'].apply(preprocess_name)
test['Extracted_name']=test['Name'].apply(preprocess_name)
train['Ticket_new']=train['Ticket'].apply(preprocess_ticket)
test['Ticket_new']=test['Ticket'].apply(preprocess_ticket)
#dropping the Name and Ticket column
train.drop(['Name'],axis=1,inplace=True)
train.drop(['Ticket'],axis=1,inplace=True)
test.drop(['Name'],axis=1,inplace=True)
test.drop(['Ticket'],axis=1,inplace=True)
#printing the train data
train
    

In [11]:
test

NameError: name 'test' is not defined

In [ ]:
#Encoding the categorical features

def label_encode(train, test, categorical_label):
    # Initialize  LabelEncoder
    le = LabelEncoder()
    # Encode categorical label 
    for x in categorical_label:
        train[x]=train[x].astype(str)
        test[x]=test[x].astype(str)
        #apply label encoder
        train[x]=le.fit_transform(train[x])
        test[x]=le.fit_transform(test[x])

    return train, test


# Assuming train and test DataFrames exist

categorical_label = ['Sex','Embarked','Extracted_name','Cabin']

train_encoded, test_encoded = label_encode(train, test, categorical_label)
train_encoded

In [ ]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader


print (torch.__version__)
print(torch.cuda.is_available())

In [ ]:
#selecting the feature and target column
X=train_encoded.iloc[:,[0] + list(range(2,11))]
print(X)
y=train_encoded.iloc[:,[1]]
print(y)

In [ ]:
#spilting train and validation data
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.2,random_state=5)
#converting to PyTorch Tensors
X_train_tens=torch.FloatTensor(X_train.values)
X_valid_tens=torch.FloatTensor(X_valid.values)
y_train_tens=torch.FloatTensor(y_train.values)
y_valid_tens=torch.FloatTensor(y_valid.values)


In [ ]:
#create datasets and dataloaders
train_dataset=TensorDataset(X_train_tens,y_train_tens)
val_dataset=TensorDataset(X_valid_tens,y_valid_tens)

train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=32)


In [ ]:
#define Neural Network 
class TitanicNN(nn.Module):
    def __init__(self,input_size):
        super(TitanicNN,self).__init__()
        self.fc1=nn.Linear(input_size,64)
        self.fc2=nn.Linear(64,32) 
        self.fc3=nn.Linear(32,2)#2 results either 1 survived or o not survived
        self.dropout=nn.Dropout(0.2)
        self.relu=nn.ReLU()
    def forward(self,x):
        x=self.relu(self.fc1(x))
        x=self.dropout(x)
        x=self.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.fc3(x)
        return x
#intialize the model
input_size=X_train.shape[1]
model=TitanicNN(input_size)
print(model)
 

In [ ]:
#define loss function and optimizer
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

In [ ]:

from tqdm import tqdm  # Progress bar for better visualization

def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to GPU if available
    
    best_accuracy=0.0
    train_losses = []
    val_losses = []
    val_accuracies = []
    # Create directory if it doesn't exist
    #os.makedirs('/kaggle/working/titanic', exist_ok=True)
    checkpoint_pth='/kaggle/working/titanic.pth'

    for epoch in range(epochs):
        # Training
        model.train()#sets the model to training mode
        running_loss = 0.0

        for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")):
            inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU/CPU where the model is

            optimizer.zero_grad()#clears previous gradient
            outputs = model(inputs)#forward pass: computes model predictions

            # Debugging Print Statements
            print(f"Batch {batch_idx}: outputs shape: {outputs.shape}, labels shape: {labels.shape}")

            labels = labels.squeeze().long() #.squeeze remove unnecessary dimensions # Convert to interger for classification
            loss = criterion(outputs, labels)#computes loss between predictions and ground truth
            loss.backward()#backward pass: computes gradients
            optimizer.step()#updates model parameters
            running_loss += loss.item()#accumulates batch loss

        train_loss = running_loss / len(train_dataloader)#computes the train loss for epoch
        train_losses.append(train_loss)#store epoch training loss

        # Validation
        model.eval()#set the model to evaluation model
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():#Disables gradient calculation
            for batch_idx, (inputs, labels) in enumerate(val_dataloader):
                inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU/CPU
                labels = labels.squeeze().long()

                outputs = model(inputs)#forward pass
                
                print(f"Validation Batch {batch_idx}: outputs shape: {outputs.shape}, labels shape: {labels.shape}")

                loss = criterion(outputs, labels)#Computes validation loss
                val_loss += loss.item()#accumulates the validation loss

                _, predicted = torch.max(outputs.data, 1)#Gets predicted class indices (argmax of outputs)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_dataloader)  # Compute average validation loss
        val_losses.append(val_loss)#store epoch validation  loss

        val_accuracy = correct / total if total > 0 else 0  # Compute validation accuracy
        val_accuracies.append(val_accuracy)#store epoch validation accuracies
        

        # Print training progress
        print(f'Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')
        if val_accuracy>best_accuracy:
           print(f"Validation accuracy improved from{val_accuracy:.4f}to{best_accuracy:.4f}")
           best_accuracy = val_accuracy
            
           #Save the best model state
           torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': best_accuracy,
                'train_loss': train_loss,
                'val_loss': val_loss,
            }, checkpoint_pth)

    return train_losses, val_losses, val_accuracies

# Train the model
train_losses, val_losses, val_accuracies = train_model(
    model, train_dataloader, val_dataloader, criterion, optimizer,epochs=98
)
torch.save(model,'titanic.pth')

In [ ]:
#Evaluate the model
pl.figure(figsize=(12,5))
pl.subplot(1,2,1)
pl.plot(train_losses,label="train loss" )
pl.plot(val_losses,label="validation loss")
pl.legend()
pl.title('loss over epochs')
#plot validation Accuracy
pl.subplot(1,2,1)
pl.plot(val_accuracies,label="validation accuracy")
pl.legend()
pl.show()


In [ ]:
# Ensure test_encoded is clean and contains only numeric values (except PassengerId)
passenger_ids = test_encoded['PassengerId']  # Save this before dropping
test_data = test_encoded.drop('PassengerId', axis=1)

# Ensure all data is numeric
test_data = test_data.apply(pd.to_numeric, errors='coerce')
test_data = test_data.fillna(0)  # Optionally use other imputation methods

# Convert to tensor
test_tensor = torch.FloatTensor(test_data.values)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(test_tensor)
    _, predictions = torch.max(outputs, 1)

# Create submission file
submission = pd.DataFrame({
    'PassengerId': passenger_ids,  # Use saved values
    'Survived': predictions.numpy()
})

submission.to_csv('/kaggle/working/titanic_submission.csv', index=False)
